In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import vgg16 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img

## Load Pre-trained Model

In [ ]:
vgg_model = vgg16.VGG16(weights='imagenet',
                        include_top=False,
                        input_shape=(224, 224, 3))

In [ ]:
vgg_model.summary()

## Prepare Dataset

## Setup The Data Generators

In [ ]:
train_dir = './datasets/train/'
validation_dir = './datasets/validation/'
 
n_train = 900 # of total 3302
n_validation = 240 # of total 824

*Notice that batch size must match with the amount of train or validation!*

In [ ]:
# change the batch size according to your system RAM
train_batch_size = 20
validation_batch_size = 20

#### Train dataset

In [ ]:
# remember that the original image size is 224 x 224 and the last layer has a shape of 7 x 7 x 512
train_datagen = ImageDataGenerator(rescale=1./255)

train_features = np.zeros(shape=(n_train, 7, 7, 512)) #(600, 7, 7, 512)
train_labels = np.zeros(shape=(n_train, 3))
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=train_batch_size,
    class_mode='categorical')

In [ ]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_model.predict(inputs_batch)
    train_features[i * train_batch_size: (i+1) * train_batch_size] = features_batch
    train_labels[i * train_batch_size : (i+1) * train_batch_size] = labels_batch
    
    i+=1
    if i * train_batch_size >= n_train:
        break

In [ ]:
train_features = np.reshape(train_features, (n_train, 7 * 7 * 512)) #(600, 25088)

#### Validation dataset

In [ ]:
# remember that the original image size is 224 x 224 anfd the last layer has a shape of 7 x 7 x 512
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_features = np.zeros(shape=(n_validation, 7, 7, 512)) #(600, 7, 7, 512)
validation_labels = np.zeros(shape=(n_validation, 3))
 
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=validation_batch_size,
    class_mode='categorical')

In [ ]:
i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_model.predict(inputs_batch)
    validation_features[i * validation_batch_size: (i+1) * validation_batch_size] = features_batch
    validation_labels[i * validation_batch_size : (i+1) * validation_batch_size] = labels_batch

    i+=1
    if i * validation_batch_size >= n_validation:
        break

In [ ]:
validation_features = np.reshape(validation_features, (n_validation, 7 * 7 * 512)) #(160, 25088)

## Create A New Model (Fully-connected Layers)

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
model_fcl = models.Sequential()
model_fcl.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
model_fcl.add(layers.Dropout(0.5))
model_fcl.add(layers.Dense(3, activation='softmax'))

## Train The Model

In [ ]:
model_fcl.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history_fcl = model_fcl.fit(train_features,
                        train_labels,
                        epochs=40,
                        batch_size=20,
                        validation_data=(validation_features, validation_labels))

## Check Performance

In [ ]:
# plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history_fcl.history['loss'],'r',linewidth=3.0)
plt.plot(history_fcl.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
plt.show()

In [ ]:
# plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history_fcl.history['acc'],'r',linewidth=3.0)
plt.plot(history_fcl.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)
plt.show()

In [ ]:
fnames = validation_generator.filenames
ground_truth = validation_generator.classes

# getting the mapping from class index to class label
label_to_index = validation_generator.class_indices
idx_to_label = dict((v,k) for k,v in label_to_index.items())

In [ ]:
predictions = model_fcl.predict_classes(validation_features)
probabilities = model_fcl.predict(validation_features)

In [ ]:
errors = np.where(predictions != np.argmax(validation_labels, axis=1))[0]
print("No of errors = {}/{} ({}%)".format(len(errors), n_validation, (n_validation-len(errors))/n_validation))

In [ ]:
for i in range(len(errors)):
    prediction_class = np.argmax(probabilities[errors[i]])
    prediction_label = idx_to_label[prediction_class]
    
    print("Original label: {}, Prediction: {}, Confidence: {:.3f}".format(fnames[errors[i]].split('/')[0],
                                                                          prediction_label,
                                                                          probabilities[errors[i]][prediction_class]))
    
    original_image = load_img("{}{}".format(validation_dir, fnames[errors[i]]))
    plt.imshow(original_image)
    plt.show()

## Save The Model

In [ ]:
# Save the model
model_fcl.save('models/fcl_transfer_learning.h5')

<hr/>